<a href="https://colab.research.google.com/github/Giffy/Mobility_AIBCN/blob/master/2_Telefonia_Cat_DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Install libraries (if required)

In [0]:
# Installs PyMongo
! python -m pip install pymongo

##  Connect to Google Drive

In [0]:
# Check is Link to Drive is OK
google = !if [ -d 'GDrive/' ]; then echo "1" ; else echo "0"; fi
if (google[0] is '0' ):
  from google.colab import drive
  drive.mount('/content/GDrive/')
!if [ -d 'GDrive/' ]; then echo "Connection to Google drive successful" ; else echo "Error to connect to Google drive"; fi

Mounted at /content/GDrive/
Connection to Google drive successful


##  Load data

In [0]:
# If source data is not available in Google Drive, it is downloaded from Open Data platform.

check_file = !if [ -f 'GDrive/My Drive/coverageCatalunya.tar.gz' ]; then echo "1" ; else echo "0"; fi


if (check_file[0] is '0' ):
  print ('Dataset not available in Google Drive: downloading dataset ... \n')
  # If datafile is not in Google Drive the download it
  # Download Open Data from 'Dades obertes catalunya': https://analisi.transparenciacatalunya.cat/browse?q=cobertura%20M%C3%B2bil
  # It dowloads a 1.7Gb csv file  
  !wget "https://analisi.transparenciacatalunya.cat/api/views/g9ma-vbt8/rows.csv?accessType=DOWNLOAD"
  !mv rows* coverageCatalunya.csv
  !tar -czvf coverageCatalunya.tar.gz coverageCatalunya.csv
  # Backup data to Google drive
  !mv coverageCatalunya.tar.gz GDrive/My\ Drive/

  
if (check_file[0] is '1' ):
  print ('Dataset available in Google Drive: inflating dataset ... ')
  !tar -xvf GDrive/My\ Drive/coverageCatalunya.tar.gz coverageCatalunya.csv

Dataset available in Google Drive: inflating dataset ... 
coverageCatalunya.csv


In [0]:
# Imports data in MongoDB
!mongodb-linux-x86_64-debian71-3.0.15/bin/mongoimport --db catalunya --collection mobile --type csv --file coverageCatalunya.csv --drop --headerline

2019-01-12T23:45:01.934+0000	connected to: localhost
2019-01-12T23:45:01.935+0000	dropping: catalunya.mobile
2019-01-12T23:45:04.931+0000	[........................] catalunya.mobile	13.0 MB/1.7 GB (0.8%)
2019-01-12T23:45:07.931+0000	[........................] catalunya.mobile	25.1 MB/1.7 GB (1.5%)
2019-01-12T23:45:10.935+0000	[........................] catalunya.mobile	35.7 MB/1.7 GB (2.1%)
2019-01-12T23:45:13.931+0000	[........................] catalunya.mobile	46.3 MB/1.7 GB (2.7%)
2019-01-12T23:45:16.931+0000	[........................] catalunya.mobile	57.8 MB/1.7 GB (3.4%)
2019-01-12T23:45:19.931+0000	[........................] catalunya.mobile	69.8 MB/1.7 GB (4.1%)
2019-01-12T23:45:22.931+0000	[#.......................] catalunya.mobile	80.9 MB/1.7 GB (4.7%)
2019-01-12T23:45:25.931+0000	[#.......................] catalunya.mobile	89.7 MB/1.7 GB (5.2%)
2019-01-12T23:45:28.931+0000	[#.......................] catalunya.mobile	101.2 MB/1.7 GB (5.9%)
2019-01-12T23:45:31.931+0000	[#....

##  Connect to MongoDB database

In [0]:
import pymongo
from pymongo import MongoClient
# uri (uniform resource identifier) defines the connection parameters 
uri = 'localhost:27017'
# start client to connect to MongoDB server 
client = MongoClient( uri )

In [0]:
# defines database and the collection to use
db = client.catalunya                           # sets the database name 
collection = db.mobile                          # sets the collection to use

##  Database Visualization

In [0]:
# prints the attributes of first document ( name of columns)
print (list (collection.find().limit(1)[1]))
print ('Total documents: ' + str(  collection.count_documents({'_id' : {'$ne': None }}) ))

['_id', 'timestamp_', 'Date', 'Hora_', 'Lat', 'Long_', 'Senyal', 'Xarxa', 'Operador', 'status', 'Desc_', 'net_type', 'speed', 'satellites', 'precision1', 'provider', 'Activitat', 'downloadSpeed', 'uploadSpeed', 'MUNICIPI', 'NOM_MUNI']
Total documents: 11744914


In [0]:
# overview of data stored in each attribute
attributes = ['Senyal', 'Xarxa', 'Operador', 'status', 'Desc_', 'net_type', 'speed', 'precision1', 'provider', 'Activitat', 'NOM_MUNI','Carrier']

for attribute in attributes:
  if (attribute != '_id'):
    print ( attribute + " "+ str( collection.distinct( attribute )))

Senyal [24, 13, 21, 18, 10, 15, 7, 11, 16, 3, 12, 4, 5, 14, 55, 9, 19, 17, 1, 8, 0, 6, 22, 20, 25, 23, 28, 27, 53, 31, 2, 30, 54, 26, 29, 61, 49, 60, 58, 50, 59, 57, 32, 56, 43, 62, 33, 52, 36, 41, 39, 51, 63, 65, 99, 47, 48, 34, 64, 42, 44, 46, 38, 35, 40, 37, 45, 98, 96, 78, 93, 87, 88, 97, 76, 86]
Xarxa ['movistar', 'orange', 'vodafone', 'MetroPCS', 'UNEFON', 'TICAE', 'yoigo', 'Telefonica Moviles Espana', 'Orange F', 'Lowi', '', 'PARLEM', 'vodafone NL', "Nom├®s trucades d'emerg├¿ncia", 'T-Mobile', 'France Telcom Espana SA', 'Sense servei.', 'Solo llamadas de emergencia', 'Lebara', 'null', 'VIVO', 'Solo emergencia', 'o2 - de', 'F SFR', 'Sin servicio', 'Voz Premium', 'Orange SP', 'Vodafone', 'vodafone IT', 'I WIND', 'TELCEL', 'Plus', 'F-Bouygues Telecom', 'MOBILAND', 'MASMovil', 'CLARO BR', 'E-Plus', 'Salt', 'France Telecom Espa├▒a SA', 'TIM', 'Telef├│nica M├│viles Espa├▒a', 'BYTEL', 'O2 - UK', 'RO Vodafone RO', 'Buscando servi├ºo', 'GurbTec', 'vodafone UK', 'FIBRACAT', 'AT&T', 'NL KP

## Data modification


In [0]:
# printing a document of database 
list(collection.find().limit(1))

[{'Activitat': 'UNKNOWN',
  'Date': '17/10/2015 01:54:12 AM',
  'Desc_': 'STATE_EMERGENCY_ONLY',
  'Hora_': '01:54:12',
  'Lat': 41.38734,
  'Long_': 2.19812,
  'MUNICIPI': 80193,
  'NOM_MUNI': 'Barcelona',
  'Operador': 'ONO',
  'Senyal': 24,
  'Xarxa': 'movistar',
  '_id': ObjectId('5c3a1ff7ec11e0f6cec056bd'),
  'downloadSpeed': '',
  'net_type': '3G',
  'precision1': 43,
  'provider': 'fused',
  'satellites': 0,
  'speed': 1.4,
  'status': 2,
  'timestamp_': 1445039652105,
  'uploadSpeed': ''}]

### Define Area of study (Data Cleansing)
Defining the area of study, we can clean data to reduce the size of the database and reduce operation times.

1- Lat: 41.453045    Long: 2.037423

2- Lat: 41.453045    Long: 2.256154
<br><br>

3- Lat: 41.338663    Long: 2.037423

4- Lat: 41.338663    Long: 2.256154

In [0]:
delete = collection.delete_many({'Long_': {"$gt" : 2.256154 }}) 
delete.deleted_count

In [0]:
delete = collection.delete_many({'Long_': {"$lt" : 2.037423 }}) 
delete.deleted_count

In [0]:
delete = collection.delete_many({'Lat': {"$gt" : 41.453045 }}) 
delete.deleted_count

In [0]:
delete = collection.delete_many({'Lat': {"$lt" : 41.338663 }}) 
delete.deleted_count

### Deletion of no relevant fields :  downloadSpeed, uploadSpeed, satellites, MUNICIPI, etc...

In [0]:
filters = {'_id' : {'$ne': None }}
fields = {'$unset': {'downloadSpeed': 1, 'uploadSpeed': 1, 'satellites': 1, 'MUNICIPI': 1, 'speed': 1,
                     'status': 1, 'Senyal':1 , 'Desc_':1, 'provider': 1, 'timestamp_': 1, 'net_type': 1,
                     'NOM_MUNI': 1, 'Operador': 1, 'Xarxa': 1, 'precision1': 1}}
 
collection.update_many( filters ,fields)

In [0]:
list( collection.find().limit(1) )

[{'Activitat': 'UNKNOWN',
  'Date': '17/10/2015 01:54:12 AM',
  'Hora_': '01:54:12',
  'Lat': 41.38734,
  'Long_': 2.19812,
  '_id': ObjectId('5c3a1ff7ec11e0f6cec056bd')}]

### Add Year, Month, Day, Weekday, Hour  fields


In [0]:
from bson.objectid import ObjectId
import datetime
import sys

def round_coord(x, base=0.005):
     return round(base*round(x/base), 4)

consulta_tmp = (collection.find({},{'_id','Date','Hora_','Lat','Long_'}))
count = 0                                       # Set progress counter to 0

num_docs = collection.count_documents({})
for item in consulta_tmp:
    pk = str(item['_id'])
    datee = str(item['Date'])
    day = int(datee.split('/')[0])
    month = int(datee.split('/')[1])
    year = int(datee.split('/')[2].split(' ')[0])
    horaa = str(item['Hora_'])
    hora = int(horaa.split(':')[0])
    weekday = datetime.datetime(year, month, day).isoweekday()
    lat = float(item['Lat'])
    long = float(item['Long_'])
    lat = round_coord(lat)
    long = round_coord(long)    
    filters = { '_id' : ObjectId(pk) }   
    fields = {'$set': {'Year': year, 'Month' : month ,'Day' : day , 'Weekday' : weekday , 'Hour' : hora, 'Latitude' : lat , 'Longitude' : long }}
    collection.update_one( filters ,fields)
    
    # Progress counter 
    count += 1 
    if (count%50):    
      b = round((count*100 / num_docs),1)
      sys.stdout.write('\r'+ str(b) + "%")

100.0%

In [0]:
filters = {'_id' : {'$ne': None }}
fields = {'$unset': {'Date' : 1, 'Lat': 1, 'Long_': 1, 'Hora_':1}}

collection.update_many( filters ,fields)

### Backup database

In [0]:
# Export to CSV
!mongodb-linux-x86_64-debian71-3.0.15/bin/mongoexport --db catalunya --collection mobile --type csv --out temp_coverageCat.csv --fields Activitat,Day,Hour,Latitude,Longitude,Month,Weekday,Year,_id

2019-01-12T18:34:47.913+0000	connected to: localhost
2019-01-12T18:35:52.437+0000	exported 2800616 records


In [0]:
collection.count_documents({})

2800616

In [0]:
# Import from CSV

!mongodb-linux-x86_64-debian71-3.0.15/bin/mongoimport --db catalunya --collection mobile --type csv --file temp_coverageCat.csv --drop --headerline

2019-01-13T00:42:52.530+0000	connected to: localhost
2019-01-13T00:42:52.530+0000	dropping: catalunya.mobile
2019-01-13T00:42:55.529+0000	[#.......................] catalunya.mobile	9.9 MB/188.5 MB (5.3%)
2019-01-13T00:42:58.528+0000	[##......................] catalunya.mobile	19.3 MB/188.5 MB (10.3%)
2019-01-13T00:43:01.530+0000	[###.....................] catalunya.mobile	28.9 MB/188.5 MB (15.3%)
2019-01-13T00:43:04.528+0000	[####....................] catalunya.mobile	38.1 MB/188.5 MB (20.2%)
2019-01-13T00:43:07.528+0000	[######..................] catalunya.mobile	47.5 MB/188.5 MB (25.2%)
2019-01-13T00:43:10.528+0000	[#######.................] catalunya.mobile	57.1 MB/188.5 MB (30.3%)
2019-01-13T00:43:13.528+0000	[########................] catalunya.mobile	66.6 MB/188.5 MB (35.3%)
2019-01-13T00:43:16.528+0000	[########................] catalunya.mobile	70.6 MB/188.5 MB (37.5%)
2019-01-13T00:43:19.528+0000	[#########...............] catalunya.mobile	77.5 MB/188.5 MB (41.1%)
2019-01-13T

## Adding reference coordenates

In [0]:
pipeline = [{ '$group':{                                   # $group , generates an agrupation of data
                  '_id' : { 'Year' : '$Year', 'Month' : '$Month', 'Day' : '$Day'}        # the id for the group is the 'Carrier' name
                  ,'Year' : {'$max' : '$Year'}           # $max , keeps the 'Carrier' name as it is
                  ,'Month' : {'$max' : '$Month'}           # $max , keeps the 'Carrier' name as it is
                  ,'Day' : {'$max' : '$Day'}           # $max , keeps the 'Carrier' name as it is
                  ,'Total' : {'$sum' : 1}                  # we generate a new key as 'Total', the value will be the $sum of carriers
    
                  }
            },
            { '$project':                                  # $project , creates a table as a result
                  { '_id' : 0,                             # this is the list of items to include, '_id' is always visible. To hide it, we need to include '_id' : 0
                    'Year' : 1,                         # to show carrier, it is included  'Carrier' : 1
                    'Month' : 1,                         # to show carrier, it is included  'Carrier' : 1
                    'Day' : 1,                         # to show carrier, it is included  'Carrier' : 1
                    'Total' : 1}                           # to show the total, it is included  'Total' : 1
            },
            { '$sort': {'Total' : -1}                      # and we sort the result by Total ammount in descendent order (-1)
            }
            ]
query = list ( collection.aggregate( pipeline ))          # finally the pipeline is sent to mongoDB with collection.aggregate

In [0]:
heatmap_scale = 1

# define reference points
# 1- Lat: 41.453045 Long: 2.037423
# 2- Lat: 41.453045 Long: 2.256154
# 3- Lat: 41.338663 Long: 2.037423
# 4- Lat: 41.338663 Long: 2.256154

activitat_coor = 'IN_VEHICLE'
latitude1 = [ 41.46]#, 41.335 ]
#longitude1 = [ 2.030, 2.259 ]
longitude1 = [2.03, 2.035, 2.04, 2.045, 2.05, 2.055, 2.06, 2.065, 2.07, 2.075, 2.08, 2.085, 2.09, 2.095, 2.1, 2.105, 2.11, 2.115, 2.12, 2.125, 2.13, 2.135, 2.14, 2.145, 2.15, 2.155, 2.16, 2.165, 2.17, 2.175, 2.18, 2.185, 2.19, 2.195, 2.2, 2.205, 2.21, 2.215, 2.22, 2.225, 2.23, 2.235, 2.24, 2.245, 2.25, 2.255, 2.26]
hours = [8 , 18]


count = 0
num_docs = len( query )*heatmap_scale*47

for i in range(heatmap_scale):
  for lat in latitude1:
    for long in longitude1:
      for item in query:
          year = int(item['Year'])
          month = int(item['Month'])
          day = int(item['Day'])
          weekday = datetime.datetime(year, month, day).isoweekday()
          for hour in hours:
            fields = {'Activitat': activitat_coor, 'Year': year, 'Month' : month ,'Day' : day , 'Weekday' : weekday , 'Hour' : hora, 'Latitude' : lat , 'Longitude' : long }
            insert_result = collection.insert_one ( fields )

          # Progress counter 
          count += 1 
          if (count%50):    
            b = round((count*100 / num_docs),1)
            sys.stdout.write('\r'+ str(b) + "%")
  i +=1

100.0%

In [0]:
heatmap_scale = 1

# define reference points
# 1- Lat: 41.453045 Long: 2.037423
# 2- Lat: 41.453045 Long: 2.256154
# 3- Lat: 41.338663 Long: 2.037423
# 4- Lat: 41.338663 Long: 2.256154

activitat_coor = 'IN_VEHICLE'
latitude1 = [ 41.335, 41.34, 41.345, 41.35, 41.355, 41.36, 41.365, 41.37, 41.375, 41.38, 41.385, 41.39, 41.395, 41.4, 41.405, 41.41, 41.415, 41.42, 41.425, 41.43, 41.435, 41.44, 41.445, 41.45, 41.455, 41.46 ]#, 41.335 ]
#longitude1 = [ 2.030, 2.259 ]
longitude1 = [2.03]#, 2.035, 2.04, 2.045, 2.05, 2.055, 2.06, 2.065, 2.07, 2.075, 2.08, 2.085, 2.09, 2.095, 2.1, 2.105, 2.11, 2.115, 2.12, 2.125, 2.13, 2.135, 2.14, 2.145, 2.15, 2.155, 2.16, 2.165, 2.17, 2.175, 2.18, 2.185, 2.19, 2.195, 2.2, 2.205, 2.21, 2.215, 2.22, 2.225, 2.23, 2.235, 2.24, 2.245, 2.25, 2.255, 2.26]
hours = [8 , 18]


count = 0
num_docs = len( query )*heatmap_scale*26

for i in range(heatmap_scale):
  for lat in latitude1:
    for long in longitude1:
      for item in query:
          year = int(item['Year'])
          month = int(item['Month'])
          day = int(item['Day'])
          weekday = datetime.datetime(year, month, day).isoweekday()
          for hour in hours:
            fields = {'Activitat': activitat_coor, 'Year': year, 'Month' : month ,'Day' : day , 'Weekday' : weekday , 'Hour' : hora, 'Latitude' : lat , 'Longitude' : long }
            insert_result = collection.insert_one ( fields )

          # Progress counter 
          count += 1 
          if (count%50):    
            b = round((count*100 / num_docs),1)
            sys.stdout.write('\r'+ str(b) + "%")
  i +=1

100.0%

In [0]:
collection.count_documents({})

2959756

In [0]:
print( len(collection.distinct('Latitude')))
print( len(collection.distinct('Longitude')))

26
47


## Export to CSV

In [0]:
# Export to CSV
!mongodb-linux-x86_64-debian71-3.0.15/bin/mongoexport --db catalunya --collection mobile --type csv --out coverageCat.csv --fields Activitat,Day,Hour,Latitude,Longitude,Month,Weekday,Year

2019-01-13T00:45:51.007+0000	connected to: localhost
2019-01-13T00:46:45.912+0000	exported 2959756 records


In [0]:
!tar -czvf coverageCatv4.tar.gz coverageCat.csv
!mv coverageCatv4.tar.gz GDrive/My\ Drive/

coverageCat.csv
